# **Albumentation – Python Library for Image Augmentation**

The performance of a deep learning model is influenced by large datasets and diversity of the dataset. But, there might be situations where the dataset is simply not large enough or diverse enough. In such cases, data augmentation is used. Data augmentation is a technique that enables you to significantly increase the diversity of data available for training models, without actually collecting new data. Although deep learning models come with inbuilt methods to augment the data, these can be inefficient or lacking some required functionality. 

In this practice session, we will learn about an augmentation package for machine learning specifically using the PyTorch framework called Albumentation.

To read about Albumentation more, please refer [this](https://analyticsindiamag.com/hands-on-guide-to-albumentation/) article.

## **Hands-on implementation of albumentation transformations**

As mentioned earlier, this library gives a wide range of transformations other than the ones commonly used in other libraries. Let us see how we can implement these transformations on one image. 

Let us start with a single transformation. I have chosen a random image from google and will perform a horizontal flip. 

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn nltk gensim tensorflow keras \
    tqdm scikit-image pillow albumentations --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
# !wget https://www.imperial.ac.uk/ImageCropToolT4/imageTool/uploaded-images/newseventsimage_1529346275459_mainnews2012_x1.jpg

In [ ]:
import random
import cv2
from matplotlib import pyplot as plt
import albumentations as A
def view_transform(image):
    plt.figure(figsize=(5, 5))
    plt.axis('off')
    plt.imshow(image)
figure = cv2.imread('newseventsimage_1529346275459_mainnews2012_x1.jpg')
figure = cv2.cvtColor(figure, cv2.COLOR_BGR2RGB)
view_transform(figure)

In [ ]:
transform = A.HorizontalFlip(p=0.5)
random.seed(7)
augmented_image = transform(image=figure)['image']
view_transform(augmented_image)

The real power of albumentation is in pipelining different transformations for the image at once. Let us implement this pipeline. I will pipeline 

> *  CLAHE: Contrast Limited Adaptive Histogram Equalization to equalize images
> * Cutout: takes out a part of the image that is not very important for classification.
> * Random rotate: rotates the image by a certain degree
> * Blur: that reduces the intensity of pixels to appear blur
> * Optical distortion: This distorts certain elements of the image.
> * ShiftScaleRotate: Allows you to scale and rotate the image by certain angles. 

In [ ]:
transform = A.Compose([
    A.CLAHE(),
    A.RandomRotate90(),
    A.Transpose(),
    A.Cutout(num_holes=1, max_h_size=16,max_w_size = 16,p=1),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
    A.Blur(blur_limit=3),
    A.OpticalDistortion(),
])
random.seed(42) 
augmented_image = transform(image=figure)['image']
view_transform(augmented_image)

As you can see, all of these transformations are applied in a pipeline and in a really quick and efficient way. 

Another interesting feature of this is called the OneOf method. Here, the transformation defined in the OneOf block is assigned with probabilities. These are normalized and the transformation with the highest normalized value is selected and applied on the image. This way, there is more efficiency in applying suitable transformations.

In [ ]:
transform = A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.3),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    ])
random.seed(42) 
augmented_image = transform(image=figure)['image']
view_transform(augmented_image)

In this example above, the one of the method has motion blur, median blur and blur with assigned probabilities. Let us normalize this to see which has the highest probability. 

Motion blur = (0.2 )/(0.2+0.3+0.1) =0.3

Median blur = (0.3)/(0.2+0.3+0.1)=0.5

Blur = (0.1)/(0.2+0.3+0.1)=0.17

The above calculations make it clear that the median blur will be applied. This way of pipelining increased the way the CPU is used. 

#**Related Articles:**

> * [Guide to Albumentation](https://analyticsindiamag.com/hands-on-guide-to-albumentation/)

> * [Google STAC](https://analyticsindiamag.com/googles-stac-ssl-framework-for-object-detection/)

> * [Point Transformers](https://analyticsindiamag.com/how-point-transformer-excels-in-3d-image-processing/)

> * [Comparison of Transfer Learning with Multi Class Classification](https://analyticsindiamag.com/practical-comparison-of-transfer-learning-models-in-multi-class-image-classification/)

> * [Fruit Recognition with CNN](https://analyticsindiamag.com/fruit-recognition-using-the-convolutional-neural-network/)

> * [Semantic Segmentation Using TensorFlow Keras](https://analyticsindiamag.com/semantic-segmentation-using-tensorflow-keras/)
